In [58]:
import pandas as pd
import numpy as np
import joblib
import json
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import smps
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
from smps.fit import LogNormal
import plotly as py
import pyarrow.feather as feather
from math import sqrt
from sklearn.metrics import mean_squared_error
sns.set("notebook", "ticks", palette='colorblind') 
%matplotlib inline

In [59]:
folder = '/Users/zahrashivji/Dropbox/Shivji/Final Data/Indoor Files/Short/Final/Shortened/'

data = dd.read_csv(
    folder + 'timeseries_forIgor_Nofogging_Shortened.csv',
    parse_dates=['timestamp']
).compute()

# Set the index to be the timestamp
data.set_index("timestamp", inplace=True) 

data

,Median (nm),Mean (nm),Mode (nm),Geo. Mean (nm),PM1_SMPS,PM_350_SMPS,PM_300_SMPS,% > 350nm,% > 300nm,PM1_modpm,...,PM1_frac_neph,PM25_frac_neph,PM10_frac_neph,PM25/PM1_neph,PM25/PM1_modpm,PMS PM1 bias,modpm PM1 bias,temp,rh,fogging_mask
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-04-08 00:00:00,49.313,68.491,47.829,55.672,1.256482,0.556169,0.603550,0.442640,0.480349,0.604283,...,0.000000,0.500000,0.500000,inf,1.161624,-1.000000,-0.519067,23.888333,29.341667,0.0
2022-04-08 00:06:00,49.614,69.711,41.418,56.214,1.309211,0.559262,0.629811,0.427175,0.481062,0.586383,...,NaN,NaN,NaN,NaN,1.185885,-1.000000,-0.552109,23.896667,29.340000,0.0
2022-04-08 00:12:00,48.548,67.842,44.508,55.173,1.289442,0.589327,0.649789,0.457040,0.503931,0.653717,...,NaN,NaN,NaN,NaN,1.149606,-1.000000,-0.493024,23.905000,29.316667,0.0
2022-04-08 00:18:00,48.714,67.580,39.954,55.239,1.146094,0.427651,0.485799,0.373138,0.423873,0.589450,...,0.000000,0.058616,0.941384,inf,1.182006,-1.000000,-0.485688,23.911667,29.318333,0.0
2022-04-08 00:24:00,48.517,68.700,44.508,55.649,1.405765,0.670301,0.735449,0.476823,0.523167,0.595883,...,0.000000,0.500000,0.500000,inf,1.125696,-1.000000,-0.576114,23.918333,29.323333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-12 06:30:00,91.212,102.960,109.411,87.283,5.649964,1.333240,1.683611,0.235973,0.297986,5.050600,...,0.263093,0.358563,0.378344,1.362874,1.016381,-0.444451,-0.106083,23.631667,37.446667,0.0
2022-04-12 06:36:00,92.069,103.713,113.419,88.095,5.730358,1.477426,1.886989,0.257824,0.329297,4.440700,...,0.250373,0.360178,0.389448,1.438566,1.023544,-0.585512,-0.225057,23.630000,37.501667,0.0
2022-04-12 06:42:00,91.502,104.038,105.545,87.560,6.214189,1.893752,2.329863,0.304746,0.374926,4.303617,...,0.257473,0.360782,0.381745,1.401242,1.015146,-0.615584,-0.307453,23.635000,37.548333,0.0


In [60]:
folder = input_raw = '/Users/zahrashivji/Dropbox/Shivji/Final Data/Indoor Files/Short/Final/Shortened/'
bin_list = ['bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10']

# group into 10
    # by RH
num_list = list(np.linspace(data['rh'].min(), data['rh'].max(), 11))
data['RH_key'] = data['rh'].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
rh_bias = data.groupby('RH_key').mean()
rh_bias.to_csv(folder + 'RH_noFogging_forIgor_Shortened.csv')

    # by SMPS mass loading
num_list = list(np.linspace(data["PM1_SMPS"].min(), data["PM1_SMPS"].max(), 11))
data['ml_key'] = data["PM1_SMPS"].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
ml_bias = data.groupby('ml_key').mean()
ml_bias.to_csv(folder + 'ml_noFogging_forIgor_Shortened.csv')

    # by SMPS geo mean diameter
num_list = list(np.linspace(data["Geo. Mean (nm)"].min(), data["Geo. Mean (nm)"].max(), 11))
data['gm_key'] = data["Geo. Mean (nm)"].apply(lambda x: bin_list[0] if ((x >= num_list[0]) & (x <= num_list[1])) \
                                      else (bin_list[1] if ((x >= num_list[1]) & (x <= num_list[2])) \
                                      else (bin_list[2] if ((x >= num_list[2]) & (x <= num_list[3])) \
                                      else (bin_list[3] if ((x >= num_list[3]) & (x <= num_list[4])) \
                                      else (bin_list[4] if ((x >= num_list[4]) & (x <= num_list[5])) \
                                      else (bin_list[5] if ((x >= num_list[5]) & (x <= num_list[6])) \
                                      else (bin_list[6] if ((x >= num_list[6]) & (x <= num_list[7])) \
                                      else (bin_list[7] if ((x >= num_list[7]) & (x <= num_list[8])) \
                                      else (bin_list[8] if ((x >= num_list[8]) & (x <= num_list[9])) \
                                      else bin_list[9])))))))))
gm_bias = data.groupby('gm_key').mean()
gm_bias.to_csv(folder + 'gm_noFogging_forIgor_Shortened.csv')